In [1]:
import numpy as np
import pandas as pd
from src.dataloader_ import *
from src.network_ import *
from src.utils import *

import os
import sys
import gc
import pickle
import numpy as np
import pandas as pd

train_series_dir = "../../inputs/series_train.parquet/"
test_series_dir = "../../inputs/series_test.parquet/"

data_dic_path = "../../inputs/data_dictionary.csv"
sample_submission_path = "../../inputs/sample_submission.csv"
train_path = "../../inputs/train.csv"
test_path = "../../inputs/test.csv"

train = pd.read_csv(train_path)
test = pd.read_csv(test_path)
sample_submission = pd.read_csv(sample_submission_path)
data_dic = pd.read_csv(data_dic_path)

import os
import random

import numpy as np
import torch


def seed_torch(seed=1029):
    random.seed(seed)
    os.environ["PYTHONHASHSEED"] = str(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.backends.cudnn.deterministic = True


nb_name = os.path.basename(os.getcwd())  # notebook name
seed_torch(seed=42)

In [2]:
from sklearn.impute import SimpleImputer, KNNImputer


def feature_engineering(df):
    # season_cols = [col for col in df.columns if "Season" in col]
    # df = df.drop(season_cols, axis=1)
    df["BMI_Age"] = df["Physical-BMI"] * df["Basic_Demos-Age"]
    df["Internet_Hours_Age"] = (
        df["PreInt_EduHx-computerinternet_hoursday"] * df["Basic_Demos-Age"]
    )
    df["BMI_Internet_Hours"] = (
        df["Physical-BMI"] * df["PreInt_EduHx-computerinternet_hoursday"]
    )
    df["BFP_BMI"] = df["BIA-BIA_Fat"] / df["BIA-BIA_BMI"]
    df["FFMI_BFP"] = df["BIA-BIA_FFMI"] / df["BIA-BIA_Fat"]
    df["FMI_BFP"] = df["BIA-BIA_FMI"] / df["BIA-BIA_Fat"]
    df["LST_TBW"] = df["BIA-BIA_LST"] / df["BIA-BIA_TBW"]
    df["BFP_BMR"] = df["BIA-BIA_Fat"] * df["BIA-BIA_BMR"]
    df["BFP_DEE"] = df["BIA-BIA_Fat"] * df["BIA-BIA_DEE"]
    df["BMR_Weight"] = df["BIA-BIA_BMR"] / df["Physical-Weight"]
    df["DEE_Weight"] = df["BIA-BIA_DEE"] / df["Physical-Weight"]
    df["SMM_Height"] = df["BIA-BIA_SMM"] / df["Physical-Height"]
    df["Muscle_to_Fat"] = df["BIA-BIA_SMM"] / df["BIA-BIA_FMI"]
    df["Hydration_Status"] = df["BIA-BIA_TBW"] / df["Physical-Weight"]
    df["ICW_TBW"] = df["BIA-BIA_ICW"] / df["BIA-BIA_TBW"]

    return df


imputer = KNNImputer(n_neighbors=5)
numeric_cols = test.select_dtypes(include=["float64", "int64"]).columns
imputed_data = imputer.fit_transform(train[numeric_cols])
train_imputed = pd.DataFrame(imputed_data, columns=numeric_cols)
# train_imputed["sii"] = train_imputed["sii"].round().astype(int)
for col in train.columns:
    if col not in numeric_cols:
        train_imputed[col] = train[col]

train = train_imputed

train = feature_engineering(train)
train = train.dropna(thresh=10, axis=0)
test = feature_engineering(test)

train_id_df = train["id"]
test_id_df = test["id"]

train = train.drop("id", axis=1)
test = test.drop("id", axis=1)


featuresCols = [
    "Basic_Demos-Age",
    "Basic_Demos-Sex",
    "CGAS-CGAS_Score",
    "Physical-BMI",
    "Physical-Height",
    "Physical-Weight",
    "Physical-Waist_Circumference",
    "Physical-Diastolic_BP",
    "Physical-HeartRate",
    "Physical-Systolic_BP",
    "Fitness_Endurance-Max_Stage",
    "Fitness_Endurance-Time_Mins",
    "Fitness_Endurance-Time_Sec",
    "FGC-FGC_CU",
    "FGC-FGC_CU_Zone",
    "FGC-FGC_GSND",
    "FGC-FGC_GSND_Zone",
    "FGC-FGC_GSD",
    "FGC-FGC_GSD_Zone",
    "FGC-FGC_PU",
    "FGC-FGC_PU_Zone",
    "FGC-FGC_SRL",
    "FGC-FGC_SRL_Zone",
    "FGC-FGC_SRR",
    "FGC-FGC_SRR_Zone",
    "FGC-FGC_TL",
    "FGC-FGC_TL_Zone",
    "BIA-BIA_Activity_Level_num",
    "BIA-BIA_BMC",
    "BIA-BIA_BMI",
    "BIA-BIA_BMR",
    "BIA-BIA_DEE",
    "BIA-BIA_ECW",
    "BIA-BIA_FFM",
    "BIA-BIA_FFMI",
    "BIA-BIA_FMI",
    "BIA-BIA_Fat",
    "BIA-BIA_Frame_num",
    "BIA-BIA_ICW",
    "BIA-BIA_LDM",
    "BIA-BIA_LST",
    "BIA-BIA_SMM",
    "BIA-BIA_TBW",
    "PAQ_A-PAQ_A_Total",
    "PAQ_C-PAQ_C_Total",
    "SDS-SDS_Total_Raw",
    "SDS-SDS_Total_T",
    "PreInt_EduHx-computerinternet_hoursday",
    "sii",
    "BMI_Age",
    "Internet_Hours_Age",
    "BMI_Internet_Hours",
    "BFP_BMI",
    "FFMI_BFP",
    "FMI_BFP",
    "LST_TBW",
    "BFP_BMR",
    "BFP_DEE",
    "BMR_Weight",
    "DEE_Weight",
    "SMM_Height",
    "Muscle_to_Fat",
    "Hydration_Status",
    "ICW_TBW",
]

# featuresCols += time_series_cols

train = train[featuresCols]
train = train.dropna(subset="sii")
train_sii_df = train["sii"]
train = train.drop("sii", axis=1)

featuresCols = [
    "Basic_Demos-Age",
    "Basic_Demos-Sex",
    "CGAS-CGAS_Score",
    "Physical-BMI",
    "Physical-Height",
    "Physical-Weight",
    "Physical-Waist_Circumference",
    "Physical-Diastolic_BP",
    "Physical-HeartRate",
    "Physical-Systolic_BP",
    "Fitness_Endurance-Max_Stage",
    "Fitness_Endurance-Time_Mins",
    "Fitness_Endurance-Time_Sec",
    "FGC-FGC_CU",
    "FGC-FGC_CU_Zone",
    "FGC-FGC_GSND",
    "FGC-FGC_GSND_Zone",
    "FGC-FGC_GSD",
    "FGC-FGC_GSD_Zone",
    "FGC-FGC_PU",
    "FGC-FGC_PU_Zone",
    "FGC-FGC_SRL",
    "FGC-FGC_SRL_Zone",
    "FGC-FGC_SRR",
    "FGC-FGC_SRR_Zone",
    "FGC-FGC_TL",
    "FGC-FGC_TL_Zone",
    "BIA-BIA_Activity_Level_num",
    "BIA-BIA_BMC",
    "BIA-BIA_BMI",
    "BIA-BIA_BMR",
    "BIA-BIA_DEE",
    "BIA-BIA_ECW",
    "BIA-BIA_FFM",
    "BIA-BIA_FFMI",
    "BIA-BIA_FMI",
    "BIA-BIA_Fat",
    "BIA-BIA_Frame_num",
    "BIA-BIA_ICW",
    "BIA-BIA_LDM",
    "BIA-BIA_LST",
    "BIA-BIA_SMM",
    "BIA-BIA_TBW",
    "PAQ_A-PAQ_A_Total",
    "PAQ_C-PAQ_C_Total",
    "SDS-SDS_Total_Raw",
    "SDS-SDS_Total_T",
    "PreInt_EduHx-computerinternet_hoursday",
    "BMI_Age",
    "Internet_Hours_Age",
    "BMI_Internet_Hours",
    "BFP_BMI",
    "FFMI_BFP",
    "FMI_BFP",
    "LST_TBW",
    "BFP_BMR",
    "BFP_DEE",
    "BMR_Weight",
    "DEE_Weight",
    "SMM_Height",
    "Muscle_to_Fat",
    "Hydration_Status",
    "ICW_TBW",
]

# featuresCols += time_series_cols
test = test[featuresCols]
test["id"] = test_id_df

# column名をfeature_iに変更

feature_cols = [f"feature_{i}" for i in range(train.shape[1])]
train.columns = feature_cols
train["id"] = train_id_df
train["sii"] = train_sii_df

In [3]:
train

,feature_0,feature_1,feature_2,feature_3,feature_4,feature_5,feature_6,feature_7,feature_8,feature_9,...,feature_55,feature_56,feature_57,feature_58,feature_59,feature_60,feature_61,feature_62,id,sii
0,5.0,0.0,51.0,16.877316,46.0,50.8,23.2,59.2,82.0,106.6,...,8591.822097,13746.944840,18.356260,29.370079,0.424811,6.383063,0.643522,0.747453,00008ff9,2.0
1,9.0,0.0,68.8,14.035590,48.0,46.0,22.0,75.0,70.0,122.0,...,3719.320478,5950.914352,20.362087,32.579348,0.321056,12.718037,0.588157,0.777492,000fd460,0.0
2,10.0,1.0,71.0,16.648696,56.5,75.6,25.0,65.0,94.0,117.0,...,13767.473069,21724.485497,12.926463,20.397407,0.347170,4.884479,0.477630,0.657942,00105258,0.0
3,9.0,0.0,71.0,18.292347,56.0,81.6,26.0,60.0,97.0,117.0,...,21298.377749,36207.411592,13.865564,23.571569,0.472854,6.274343,0.563684,0.661008,00115b9f,1.0
5,13.0,1.0,50.0,22.279952,59.5,112.2,30.8,60.0,73.0,102.0,...,90468.027355,135701.701175,11.862478,17.793672,0.594629,2.621003,0.562625,0.521399,001f3379,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3953,8.0,0.0,74.2,17.139810,52.5,67.2,25.0,60.0,65.0,112.0,...,15184.201563,25813.157324,15.405804,26.189881,0.385990,5.416433,0.546400,0.700249,ff6c2bb8,0.0
3954,7.0,1.0,59.4,13.927006,48.5,46.6,23.0,65.0,75.0,105.0,...,1367.006219,1777.103699,20.735773,26.956438,0.373066,43.676843,0.644749,0.667565,ff759544,1.0
3955,13.0,0.0,60.0,16.362460,59.5,82.4,25.0,71.0,70.0,104.0,...,13985.687504,23775.715110,14.646602,24.899272,0.500487,12.939628,0.641165,0.631642,ff8a2de4,1.0
3957,11.0,0.0,68.0,21.441500,60.0,109.8,29.2,79.0,99.0,116.0,...,42623.022658,68197.040233,11.418397,18.269490,0.479653,4.334530,0.496020,0.623919,ffcd4dbd,1.0


In [4]:
from sklearn.preprocessing import StandardScaler, MinMaxScaler

scaler = StandardScaler()
# scaler = MinMaxScaler()

# 各列のinf, -infを各列における最大値、最小値に変換
train = train.replace([np.inf, -np.inf], np.nan)
train = train.fillna(train.max())

train[feature_cols] = scaler.fit_transform(train[feature_cols].values)

with open("./assets/scaler.pkl", "wb") as f:
    pickle.dump(scaler, f)

In [5]:
train_df = train

### テーブルデータセット

In [6]:
# # onehotEncoderの作成
# from sklearn.preprocessing import OneHotEncoder

# categorical_columns = [
#     "Basic_Demos-Enroll_Season",
#     "CGAS-Season",
#     "Physical-Season",
#     "PAQ_C-Season",
#     "FGC-Season",
#     "Fitness_Endurance-Season",
#     "PAQ_A-Season",
#     "BIA-Season",
#     "SDS-Season",
#     "PreInt_EduHx-Season",
# ]

# double_columns = [
#     "FGC-FGC_SRR_Zone",
#     "BIA-BIA_SMM",
#     "Physical-Waist_Circumference",
#     "BIA-BIA_FFMI",
#     "FGC-FGC_CU",
#     "PreInt_EduHx-computerinternet_hoursday",
#     "BIA-BIA_ECW",
#     "FGC-FGC_CU_Zone",
#     "FGC-FGC_SRL_Zone",
#     "BIA-BIA_DEE",
#     "Physical-Weight",
#     "Fitness_Endurance-Time_Mins",
#     "FGC-FGC_SRR",
#     "SDS-SDS_Total_T",
#     "FGC-FGC_PU",
#     "BIA-BIA_FFM",
#     "FGC-FGC_TL_Zone",
#     "Physical-BMI",
#     "Physical-Systolic_BP",
#     "Physical-HeartRate",
#     "BIA-BIA_ICW",
#     "Physical-Height",
#     "FGC-FGC_SRL",
#     "BIA-BIA_BMC",
#     "Fitness_Endurance-Time_Sec",
#     "BIA-BIA_Frame_num",
#     "Basic_Demos-Age",
#     "FGC-FGC_GSND_Zone",
#     "Basic_Demos-Sex",
#     "FGC-FGC_GSND",
#     "BIA-BIA_LST",
#     "FGC-FGC_TL",
#     "BIA-BIA_BMI",
#     "BIA-BIA_FMI",
#     "PAQ_C-PAQ_C_Total",
#     "BIA-BIA_Activity_Level_num",
#     "FGC-FGC_GSD",
#     "BIA-BIA_BMR",
#     "BIA-BIA_Fat",
#     "SDS-SDS_Total_Raw",
#     "CGAS-CGAS_Score",
#     "FGC-FGC_PU_Zone",
#     "BIA-BIA_LDM",
#     "Fitness_Endurance-Max_Stage",
#     "PAQ_A-PAQ_A_Total",
#     "BIA-BIA_TBW",
#     "FGC-FGC_GSD_Zone",
#     "Physical-Diastolic_BP",
# ]

# ###################### categorical columns ######################
# # trainのtargetをonehot化
# onehot_encoder = OneHotEncoder(handle_unknown="ignore", sparse=False)
# onehot_encoder.fit(train[categorical_columns])

# with open("./assets/onehot_encoder.pkl", "wb") as f:
#     pickle.dump(onehot_encoder, f)

# categorical_feature = onehot_encoder.transform(train[categorical_columns])

# ###################### double columns ######################
# # trainのtargetを標準化
# from sklearn.preprocessing import StandardScaler

# scaler = StandardScaler()
# scaler.fit(train[double_columns + add_features])

# with open("./assets/scaler.pkl", "wb") as f:
#     pickle.dump(scaler, f)

# double_feature = scaler.transform(train[double_columns + add_features])
# # double_feature = train[double_columns].values

# # 欠損値の補完
# double_feature = np.nan_to_num(double_feature)

# ###################### inputの作成 ######################

# ids = train["id"].values.reshape(-1, 1)
# X = np.concatenate([categorical_feature, double_feature], axis=1)
# y = train["sii"].fillna(-1).values.reshape(-1, 1)

# # DataFrameの作成
# ids_df = pd.DataFrame(ids, columns=["id"])
# X_df = pd.DataFrame(X, columns=[f"feature_{i}" for i in range(X.shape[1])])
# y_df = pd.DataFrame(y, columns=["sii"])

# train_df = pd.concat([ids_df, X_df, y_df], axis=1)
# train_df

In [7]:
def read_parquet(base_dir, id_):
    path = os.path.join(base_dir, f"id={id_}", "part-0.parquet")
    return pd.read_parquet(path)


def get_valid_ids(base_dir):
    return [f.split("=")[1].split(".")[0] for f in os.listdir(base_dir)]


p = read_parquet(base_dir="../../inputs/series_train.parquet/", id_="ffcd4dbd")
# p = read_parquet(base_dir="../../inputs/series_train.parquet/", id_="10e46254")
# p

In [8]:
from glob import glob

# len(glob("../../normalized/*"))
len(glob("../../inputs/series_train.parquet/*"))

996

## Metric

In [9]:
from sklearn.metrics import *


def quadratic_weighted_kappa(y_true, y_pred):
    return cohen_kappa_score(y_true, y_pred, weights="quadratic")


def threshold_Rounder(oof_non_rounded, thresholds):
    return np.where(
        oof_non_rounded < thresholds[0],
        0,
        np.where(
            oof_non_rounded < thresholds[1],
            1,
            np.where(oof_non_rounded < thresholds[2], 2, 3),
        ),
    )


def evaluate_predictions(thresholds, y_true, oof_non_rounded):
    rounded_p = threshold_Rounder(oof_non_rounded, thresholds)
    return -quadratic_weighted_kappa(y_true, rounded_p)

## Model, Dataset

In [10]:
train_df.head()

,feature_0,feature_1,feature_2,feature_3,feature_4,feature_5,feature_6,feature_7,feature_8,feature_9,...,feature_55,feature_56,feature_57,feature_58,feature_59,feature_60,feature_61,feature_62,id,sii
0,-1.528487,-0.757178,-0.734160,-0.364967,-1.357717,-0.857410,-0.845989,-0.797614,0.009947,-0.628279,...,0.018156,0.017977,-0.120290,-0.123103,-0.115835,-0.024720,-0.137810,1.628880,00008ff9,2.0
1,-0.361407,-0.757178,0.074162,-0.895784,-1.084807,-0.969239,-1.110982,0.396247,-0.891203,0.301031,...,0.017806,0.017604,-0.107931,-0.109903,-0.194942,0.163257,-0.142769,2.093620,000fd460,0.0
2,-0.069637,1.320694,0.174067,-0.407672,0.075057,-0.279626,-0.448501,-0.359361,0.911097,-0.000693,...,0.018528,0.018360,-0.153747,-0.160009,-0.175032,-0.069187,-0.152671,0.244050,00105258,0.0
3,-0.361407,-0.757178,0.174067,-0.100648,0.006830,-0.139839,-0.227674,-0.737165,1.136384,-0.000693,...,0.019070,0.019054,-0.147961,-0.146954,-0.079206,-0.027946,-0.144962,0.291482,00115b9f,1.0
5,0.805674,1.320694,-0.779571,0.644211,0.484422,0.573073,0.832296,-0.737165,-0.665916,-0.905865,...,0.024046,0.023826,-0.160303,-0.170719,0.013641,-0.136351,-0.145057,-1.868410,001f3379,1.0


In [11]:
from sklearn.model_selection import train_test_split

use_ids = list(
    train_df[train_df["sii"] != -1]["id"].unique()
)  # get_valid_ids(base_dir="../../normalized/")

len(use_ids)

2736

## Training

In [12]:
from tqdm import tqdm
from sklearn.model_selection import KFold

kf = KFold(n_splits=5, shuffle=True, random_state=42)
use_ids = np.array(use_ids)
for train_index, valid_index in kf.split(use_ids):
    train_ids = [use_ids[i] for i in train_index]
    valid_ids = [use_ids[i] for i in valid_index]

    train_dataset = CMIDataset(
        table_df=train_df,
        valid_ids=use_ids,
        base_dir="../../inputs/series_train.parquet/",
        save_filename=nb_name,
    )

In [13]:
train_dataset[0]["time_input"].shape

torch.Size([31, 17280, 15])

In [14]:
from tqdm import tqdm
from sklearn.model_selection import StratifiedKFold

skf = StratifiedKFold(n_splits=4, shuffle=True, random_state=42)

CV = []

# use_ids = np.array(use_ids[:30]) # debug
use_ids = np.array(use_ids)

extract_df = train[train["id"].isin(use_ids)].reset_index(drop=True)

test_df = train[["id", "sii"]].copy()
# test_df["pred_sii"] = 0
oof_preds = []

for fold, (train_ids, valid_ids) in enumerate(
    skf.split(extract_df["id"], extract_df["sii"])
):
    print(f"################### fold:{fold} ###################")
    best_valid_score = -100

    train_ids = use_ids[train_ids]
    valid_ids = use_ids[valid_ids]

    train_dataset = CMIDataset(
        table_df=train_df,
        valid_ids=train_ids,
        base_dir="../../inputs/series_train.parquet/",
        save_filename=nb_name,
    )
    train_loader = DataLoader(train_dataset, batch_size=1, shuffle=True, num_workers=30)

    vlaid_dataset = CMIDataset(
        table_df=train_df,
        valid_ids=valid_ids,
        base_dir="../../inputs/series_train.parquet/",
        save_filename=nb_name,
    )

    valid_loader = DataLoader(
        vlaid_dataset, batch_size=1, shuffle=False, num_workers=30
    )
    # data_loader = DataLoader(dataset, batch_size=1, shuffle=True)

    # model = TimeEncoder(input_size=26, hidden_size=13, num_layers=2).to("cuda")
    model = CMIModel(input_size=26, hidden_size=13, num_layers=2).to("cuda")

    criterion = nn.MSELoss()
    optimizer = torch.optim.Adam(model.parameters(), lr=0.001)
    spot_oof_preds = []

    for epoch in range(5):
        total_train_loss = 0
        total_valid_loss = 0

        train_pred = []
        valid_pred = []
        trian_gt = []
        valid_gt = []

        tq = tqdm(train_loader)
        for i, data in enumerate(train_loader):
            table_input = data["table_input"].to("cuda")
            time_input = data["time_input"].to("cuda")
            mask = data["mask"].to("cuda").to(torch.float32)
            target_ = data["output"].to("cuda")
            optimizer.zero_grad()
            output, attention_weight = model(table_input, time_input, active_mask=mask)
            loss = criterion(output, target_)
            loss.backward()
            optimizer.step()
            total_train_loss += loss.item()

            train_pred.append(output.detach().cpu().numpy())
            trian_gt.append(target_.detach().cpu().numpy())

            tq.set_postfix(loss=total_train_loss / (i + 1))
            tq.update()
        tq.close()

        tq = tqdm(valid_loader)
        for i, data in enumerate(valid_loader):
            table_input = data["table_input"].to("cuda")
            time_input = data["time_input"].to("cuda")
            mask = data["mask"].to("cuda").to(torch.float32)
            target_ = data["output"].to("cuda")
            output, attention_weight = model(table_input, time_input, active_mask=mask)
            loss = criterion(output, target_)
            total_valid_loss += loss.item()

            valid_pred.append(output.detach().cpu().numpy())
            valid_gt.append(target_.detach().cpu().numpy())

            tq.set_postfix(loss=total_valid_loss / (i + 1))
            tq.update()
        tq.close()

        metric_train_pred = np.concatenate(train_pred)
        metric_valid_pred = np.concatenate(valid_pred)
        metric_train_gt = np.concatenate(trian_gt)
        metric_valid_gt = np.concatenate(valid_gt)

        train_score = quadratic_weighted_kappa(
            metric_train_gt, metric_train_pred.round(0).astype(int)
        )

        valid_score = quadratic_weighted_kappa(
            metric_valid_gt, metric_valid_pred.round(0).astype(int)
        )

        print(
            f"epoch: {epoch}, loss: {total_train_loss / len(train_loader)}, valid_loss: {total_valid_loss / len(valid_loader)}, train_score: {train_score}, valid_score: {valid_score}"
        )

        if valid_score > best_valid_score:
            best_valid_score = valid_score
            torch.save(model.state_dict(), f"./assets/model_{fold}.pth")

            spot_oof_preds = []
            for i, id_ in enumerate(valid_ids):
                spot_oof_preds.append({"id": id_, "pred_sii": valid_pred[i][0][0]})

    oof_preds.append(spot_oof_preds)
    CV.append(best_valid_score)

print(f"CV: {np.mean(CV)}")

################### fold:0 ###################


100%|██████████| 684/684 [00:16<00:00, 42.69it/s, loss=0.465]


epoch: 0, loss: 0.5738016171157034, valid_loss: 0.4648505364448938, train_score: 0.2544443938363533, valid_score: 0.3656972773005339


100%|██████████| 684/684 [00:15<00:00, 42.94it/s, loss=0.652]


epoch: 1, loss: 0.48619254966693154, valid_loss: 0.6515343545591075, train_score: 0.37251168399233936, valid_score: 0.3665006764749634


...

epoch: 2, loss: 0.46138592571697945, valid_loss: 0.5278600387989622, train_score: 0.40452420068242134, valid_score: 0.3742137645525033


100%|██████████| 684/684 [00:15<00:00, 42.87it/s, loss=0.5]  


epoch: 3, loss: 0.4359103562776124, valid_loss: 0.4999167435236088, train_score: 0.43515345129724115, valid_score: 0.330245360641667


100%|██████████| 684/684 [00:15<00:00, 45.50it/s, loss=0.508]


epoch: 4, loss: 0.42056425620681664, valid_loss: 0.5082904064355134, train_score: 0.4766412572466552, valid_score: 0.3890280755223803
CV: 0.39156681223368395


In [15]:
train_df

,feature_0,feature_1,feature_2,feature_3,feature_4,feature_5,feature_6,feature_7,feature_8,feature_9,...,feature_55,feature_56,feature_57,feature_58,feature_59,feature_60,feature_61,feature_62,id,sii
0,-1.528487,-0.757178,-0.734160,-0.364967,-1.357717,-0.857410,-0.845989,-0.797614,0.009947,-0.628279,...,0.018156,0.017977,-0.120290,-0.123103,-0.115835,-0.024720,-0.137810,1.628880,00008ff9,2.0
1,-0.361407,-0.757178,0.074162,-0.895784,-1.084807,-0.969239,-1.110982,0.396247,-0.891203,0.301031,...,0.017806,0.017604,-0.107931,-0.109903,-0.194942,0.163257,-0.142769,2.093620,000fd460,0.0
2,-0.069637,1.320694,0.174067,-0.407672,0.075057,-0.279626,-0.448501,-0.359361,0.911097,-0.000693,...,0.018528,0.018360,-0.153747,-0.160009,-0.175032,-0.069187,-0.152671,0.244050,00105258,0.0
3,-0.361407,-0.757178,0.174067,-0.100648,0.006830,-0.139839,-0.227674,-0.737165,1.136384,-0.000693,...,0.019070,0.019054,-0.147961,-0.146954,-0.079206,-0.027946,-0.144962,0.291482,00115b9f,1.0
5,0.805674,1.320694,-0.779571,0.644211,0.484422,0.573073,0.832296,-0.737165,-0.665916,-0.905865,...,0.024046,0.023826,-0.160303,-0.170719,0.013641,-0.136351,-0.145057,-1.868410,001f3379,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3953,-0.653177,-0.757178,0.319383,-0.315935,-0.470761,-0.475327,-0.448501,-0.737165,-1.266682,-0.302417,...,0.018630,0.018556,-0.138470,-0.136184,-0.145434,-0.053402,-0.146510,0.898583,ff6c2bb8,0.0
3954,-0.944947,1.320694,-0.352705,-0.916066,-1.016580,-0.955261,-0.890155,-0.359361,-0.515724,-0.724830,...,0.017636,0.017403,-0.105628,-0.133031,-0.155288,1.081891,-0.137700,0.392937,ff759544,1.0
3955,0.805674,-0.757178,-0.325458,-0.461139,0.484422,-0.121201,-0.448501,0.094004,-0.891203,-0.785175,...,0.018544,0.018458,-0.143148,-0.141492,-0.058136,0.169832,-0.138021,-0.162838,ff8a2de4,1.0
3957,0.222133,-0.757178,0.037833,0.487594,0.552649,0.517158,0.478973,0.698490,1.286576,-0.061037,...,0.020604,0.020589,-0.163040,-0.168762,-0.074021,-0.085505,-0.151023,-0.282318,ffcd4dbd,1.0


In [16]:
oof_preds_df = pd.concat([pd.DataFrame(p) for p in oof_preds], axis=0).reset_index(
    drop=True
)
oof_preds_df.head()

,id,pred_sii
0,00105258,0.649819
1,00115b9f,0.401202
2,0038ba98,0.493791
3,00ae59c9,0.916728
4,00f574e9,1.957120


In [17]:
test_pred_df = test_df.merge(oof_preds_df, on="id", how="inner")
test_pred_df.head()

,id,sii,pred_sii
0,00008ff9,2.0,0.381639
1,000fd460,0.0,0.104846
2,00105258,0.0,0.649819
3,00115b9f,1.0,0.401202
4,001f3379,1.0,0.534297


In [18]:
from scipy.optimize import minimize

KappaOPtimizer = minimize(
    evaluate_predictions,
    x0=[0.5, 1.5, 2.5],
    args=(test_pred_df["sii"], test_pred_df["pred_sii"]),
    method="Nelder-Mead",
)
assert KappaOPtimizer.success, "Optimization did not converge."

oof_tuned = threshold_Rounder(test_pred_df["pred_sii"], KappaOPtimizer.x)
tKappa = quadratic_weighted_kappa(test_pred_df["sii"], Zoof_tuned)
print(f"tuned Kappa: {tKappa}")

tuned Kappa: 0.4361318738190414


In [19]:
print(KappaOPtimizer.x)

[0.60395977 1.10821229 2.71399198]


In [20]:
import torch
import torch.nn as nn


class Decoder(nn.Module):
    def __init__(self):
        super(Decoder, self).__init__()
        self.deconv1 = nn.ConvTranspose1d(
            32, 64, kernel_size=4, stride=2, padding=1
        )  # 時間方向に拡大
        self.deconv2 = nn.ConvTranspose1d(64, 32, kernel_size=4, stride=2, padding=1)
        self.deconv3 = nn.ConvTranspose1d(32, 15, kernel_size=4, stride=2, padding=1)

        self.upsample = nn.Upsample(size=17280, mode="linear")  # 最終的な長さに調整

    def forward(self, x):
        batch, days, time, channels = x.shape
        x = x.view(batch * days, channels, time)  # (batch*days, 32, 10)

        x = self.deconv1(x)
        x = self.deconv2(x)
        x = self.deconv3(x)

        x = self.upsample(x)  # (batch*days, 15, 17280)
        x = x.view(batch, days, 17280, 15)

        return x


# テスト
encoder_output = torch.randn(1, 31, 10, 32)
decoder = Decoder()
output = decoder(encoder_output)
print(output.shape)  # (1, 31, 17280, 15)

torch.Size([1, 31, 17280, 15])
